In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.tools.requests.tool import RequestsGetTool, TextRequestsWrapper
from langchain.agents import AgentType, initialize_agent

from pydantic import BaseModel, Field, root_validator

import tldextract
from typing import Any, Dict

## Setting the LLM

In [ ]:
chat_llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0,
    openai_api_key = open("openai_api.txt", "r").read()
)


## Tool Input Schema

By default, tools infer the argument schema by inspecting the function signature. For more strict requirements, custom input schema can be specified, along with custom validation logic.

In [ ]:
_APPROVED_DOMAINS = {
    "langchain",
    "wikipedia"
}

class ToolInputSchema(BaseModel):
    url: str = Field(...)

    @root_validator
    def validate_query(cls, values: Dict[str, Any]) -> Dict:
        url = values["url"]
        domain = tldextract.extract(url).domain
        if domain not in _APPROVED_DOMAINS:
            raise ValueError(f"Domain {domain} is not on the approved list:"
                             f" {sorted(_APPROVED_DOMAINS)}")
        return values

In [ ]:
tool = RequestsGetTool(args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper())

In [ ]:
agent = initialize_agent(
    tools = [tool],
    llm = chat_llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = False
)

In [ ]:
answer = agent.run("What's the main title on langchain.com?")
print(answer)

In [ ]:
try:
    print(agent.run("What's the main title on google.com?"))
except ValueError as e:
    print(e)